In [1]:
import pandas as pd
import numpy as np
import requests 

import warnings
warnings.filterwarnings('ignore')

In [2]:
url = 'https://analytics.deacero.com/api/teenus/get-data'
urlfin = 'format=json'

## Lectura de información

In [3]:
req_pas16= requests.get(f'{url}/3689da48-d557-5e5f-8347-006ced354939?{urlfin}').json()
req_pas17 = requests.get(f'{url}/2a323bb8-0a6d-5bd5-8366-90041c4f1c8c?{urlfin}').json()
req_vuelos16 = requests.get(f'{url}/2743ebad-f1e2-5eff-8c4d-f8c5191d1775?{urlfin}').json()
req_vuelos17 = requests.get(f'{url}/a6960833-d5a3-56dc-b125-da9e4e1fce69?{urlfin}').json()
req_aerolineas = requests.get(f'{url}/fed214f3-332d-522c-97ac-da395a066dba?{urlfin}').json()


In [4]:
pasajeros_16 = pd.DataFrame(req_pas16)
pasajeros_17 = pd.DataFrame(req_pas17)
vuelos_16 = pd.DataFrame(req_vuelos16)
vuelos_17 = pd.DataFrame(req_vuelos17)
aerolineas = pd.DataFrame(req_aerolineas)

## Exploración de Pasajeros

In [5]:
pasajeros_16.head()

,ID_Pasajero,Pasajero,Edad
0,576,Danielle Thompson,60
1,579,Natalie Cuevas,49
2,683,John Murray,28
3,681,Michael Jacobs,24
4,592,Brian Hunt,40


In [6]:
pasajeros_17.head()

,ID_Pasajero,Pasajero,Edad
0,596,Javier Olson,71
1,625,Monique Ramirez,35
2,637,Rob Beeghly,29
3,730,Timothy Moore,21
4,682,Scot Wooten,72


#### Las tablas *Pasajeros 2016* y *Pasajeros 2017* cuentan con el mismo número de columnas y tipo de dato, por lo tanto se procede a realizar una concatenación simple.

In [7]:
#Consolidacion y exploracion de Pasajeros 
pasajeros = pd.concat([pasajeros_16,pasajeros_17])

In [8]:
def explora(df):
    print('Info inicial DataFrame')
    print(df.info())
    print('------------------')
    print('Renglones y Columnas')
    print(df.shape)
    print('------------------')
    print('Total de NaN en cada columna')
    print(df.isnull().sum())
    


In [9]:
explora(pasajeros)

Info inicial DataFrame
<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 99
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID_Pasajero  200 non-null    int64 
 1   Pasajero     200 non-null    object
 2   Edad         200 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 6.2+ KB
None
------------------
Renglones y Columnas
(200, 3)
------------------
Total de NaN en cada columna
ID_Pasajero    0
Pasajero       0
Edad           0
dtype: int64


#### Ahora se busca que no haya IDs de pasajeros repetidos.

In [10]:
pasajeros['ID_Pasajero'].value_counts()

682    12
38     10
693     6
582     6
637     5
       ..
630     1
695     1
610     1
650     1
696     1
Name: ID_Pasajero, Length: 133, dtype: int64

#### Se puede ver que si existen IDs repetidos. Antes de descartarlos, primero verificamos si se utilizó el mismo ID para diferentes pasajeros.

In [11]:
# groupby ID_Pasajero y regreso booleano si hay mas de 1 pasajero por ID 
pasajeros_multiple = pasajeros.groupby(["ID_Pasajero"]).Pasajero.nunique().gt(1)

# regresa solo los valores que tiene TRUE en pasajeros_multiple, resultando los pasajeros que tienen mismo ID pero diferente informacion
pasajeros_diferentes= pasajeros.loc[pasajeros.ID_Pasajero.isin(pasajeros_multiple[pasajeros_multiple].index)]

In [12]:
pasajeros_diferentes.sort_values(by=['ID_Pasajero'])

,ID_Pasajero,Pasajero,Edad
62,562,Zachary White,49
65,562,Edward Klein,47
59,570,Erik Wheeler,30
15,570,Erik Wheeler,30
44,570,Joshua Smith,49
87,570,Erik Wheeler,30
86,570,Erik Wheeler,30
90,582,Brandon Harris,68
31,582,Brandon Harris,68
32,582,Jesse Melton,23


#### Para no descartar la información de los Pasajeros que fueron guardados con el mismo ID, la exportaremos en CSV para que el problema se pueda resolver manualmente.

In [13]:
#Exporto frame a CSV para que la informacion quede guardada y se resuelva el problema manualmente
pasajeros_diferentes.to_csv('ID_Pasajeros_repetidos.csv')

In [14]:
#Guardo los id para pasar a excluirlos 
id_pasajerosdif = pasajeros_diferentes['ID_Pasajero'].unique()

pasajeros = pasajeros[pasajeros["ID_Pasajero"].isin(id_pasajerosdif) == False]

#### Una vez teniendo una copia de duplicados en csv, se eliminan los IDs y se verifica que se hayan elimanado correctamente.

In [15]:
#Ya guardada la informacion, ya se eliminan renglones con IDs duplicados
pasajeros.drop_duplicates(inplace=True)

pasajeros["ID_Pasajero"].value_counts()

576    1
579    1
692    1
659    1
577    1
      ..
610    1
650    1
605    1
645    1
715    1
Name: ID_Pasajero, Length: 128, dtype: int64

## Exploración de Vuelos

In [16]:
vuelos_16.head()

,Cve_LA,Viaje,Clase,Precio,Ruta,Cve_Cliente
0,SW,9/10/2016,Economy,60,DAL-MDW,553
1,AA,6/11/2016,Economy,150,DAL-SLC,554
2,DA,9/21/2016,Economy,68,DAL-AMA,554
3,UA,6/28/2016,Business,160,DAL-SLC,556
4,SW,6/25/2016,Economy,65,DAL-OKC,557


In [17]:
vuelos_17.head()

,Cve_LA,Viaje,Clase,Precio,Ruta,Cve_Cliente
0,SW,1/4/2017,Economy,86,DAL-ATL,637
1,SW,1/4/2017,First Class,237,DAL-ATL,402
2,SW,1/2/2017,Economy,62,DAL-LGA,191
3,SW,1/2/2017,Economy,81,DAL-MDW,191
4,AA,1/4/2017,Business,169,DAL-LGA,637


#### Al igual que la tabla de *Pasajeros*, las tablas de vuelos cuentan con las mismas columnas, por lo tanto se hace una concatenación sencilla.

In [18]:
vuelos = pd.concat([vuelos_16,vuelos_17])

In [19]:
explora(vuelos)

Info inicial DataFrame
<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 0 to 199
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Cve_LA       400 non-null    object
 1   Viaje        400 non-null    object
 2   Clase        400 non-null    object
 3   Precio       400 non-null    int64 
 4   Ruta         400 non-null    object
 5   Cve_Cliente  400 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 21.9+ KB
None
------------------
Renglones y Columnas
(400, 6)
------------------
Total de NaN en cada columna
Cve_LA         0
Viaje          0
Clase          0
Precio         0
Ruta           0
Cve_Cliente    0
dtype: int64


In [20]:
vuelos

,Cve_LA,Viaje,Clase,Precio,Ruta,Cve_Cliente
0,SW,9/10/2016,Economy,60,DAL-MDW,553
1,AA,6/11/2016,Economy,150,DAL-SLC,554
2,DA,9/21/2016,Economy,68,DAL-AMA,554
3,UA,6/28/2016,Business,160,DAL-SLC,556
4,SW,6/25/2016,Economy,65,DAL-OKC,557
...,...,...,...,...,...,...
195,SW,8/16/2017,Economy,229,DAL-TUL,627
196,AM,6/5/2017,First Class,85,DAL-AMA,669
197,AM,6/2/2017,First Class,55,DAL-TUL,693
198,AV,2/9/2017,Business,140,DAL-MDW,637


#### Se intenta buscar que no haya información duplicada en *Vuelos*, sin embargo, al no tener un ID para los vuelos, la tarea se vuelve un poco "complicada". Se crea un ID temporal juntando la informacion de la clave de la aerolinea, fecha del vuelo, ruta, y la clave del cliente.

In [21]:
#prueba_rep =(vuelos_17.assign(Date=pd.to_datetime(vuelos_17['Viaje']).dt.normalize(),
#           ID=vuelos_16['Cve_Cliente'].duplicated(keep=False).astype(int)
#          )
#   .groupby(['Date', 'Cve_Cliente'], as_index=False).agg(repetitions=('ID', 'sum'))
#)
#prueba_rep

In [22]:
vuelos['id_temporal'] = vuelos['Cve_LA']+'_'+vuelos['Viaje']+'_'+vuelos['Ruta']+'_'+vuelos['Cve_Cliente'].astype(str)



In [23]:
len(vuelos['id_temporal'].unique())

399

#### La tabla *Vuelos* tiene un total de 400 observaciones, por lo tanto existe 1 observación duplicada (con el criterio antes mencionado).

In [24]:
duplicateIDtemp =vuelos[vuelos.duplicated(['id_temporal'])]
duplicateIDtemp

,Cve_LA,Viaje,Clase,Precio,Ruta,Cve_Cliente,id_temporal
130,AM,10/10/2017,Economy,268,DAL-PHX,582,AM_10/10/2017_DAL-PHX_582


In [25]:
prueba = vuelos.loc[vuelos['Cve_Cliente'] == 582]
prueba.sort_values(by=['Viaje'])

,Cve_LA,Viaje,Clase,Precio,Ruta,Cve_Cliente,id_temporal
107,AV,1/10/2017,First Class,69,DAL-HOU,582,AV_1/10/2017_DAL-HOU_582
81,SW,1/29/2017,Business,61,DAL-HOU,582,SW_1/29/2017_DAL-HOU_582
105,AM,10/10/2017,First Class,119,DAL-PHX,582,AM_10/10/2017_DAL-PHX_582
130,AM,10/10/2017,Economy,268,DAL-PHX,582,AM_10/10/2017_DAL-PHX_582
82,AV,10/12/2017,First Class,112,DAL-OKC,582,AV_10/12/2017_DAL-OKC_582
129,AV,11/17/2017,First Class,75,DAL-MDW,582,AV_11/17/2017_DAL-MDW_582
154,AA,11/7/2017,Business,65,DAL-SLC,582,AA_11/7/2017_DAL-SLC_582
55,AV,5/13/2017,Economy,250,DAL-GRR,582,AV_5/13/2017_DAL-GRR_582
102,AV,6/15/2017,Economy,62,DAL-ATL,582,AV_6/15/2017_DAL-ATL_582
140,AM,8/26/2017,Business,61,DAL-SLC,582,AM_8/26/2017_DAL-SLC_582


#### Al buscar la información del cliente "582" con información duplicada, se observa que lo único que cambia del vuelo ,con fecha de 10/10/2017, es la Clase y por lo tanto el precio. Aquí es necesario validar la captura, o si en efecto el, cliente hizo dos viajes el mismo día. El eliminar esta información del pasajero repercutaría en el análisis final  del precio de los vuelos, por tal razón, se decide que se mantendrá la información por el momento.

In [26]:
vuelos= vuelos.rename(columns = {'Cve_Cliente': 'ID_Pasajero'})
vuelos= vuelos.drop('id_temporal', axis=1)

#### Debido a que se desea tener solo la información existente entre el vuelo y el pasajero, se decide hacer un INNER JOIN


In [27]:
vuelos_pasajeros = pd.merge(pasajeros,vuelos, how="inner", on= ['ID_Pasajero'])
vuelos_pasajeros

,ID_Pasajero,Pasajero,Edad,Cve_LA,Viaje,Clase,Precio,Ruta
0,576,Danielle Thompson,60,AM,1/5/2016,Economy,252,DAL-SLC
1,576,Danielle Thompson,60,AM,11/25/2016,Business,269,DAL-TUL
2,576,Danielle Thompson,60,AA,9/6/2016,Business,186,DAL-AMA
3,576,Danielle Thompson,60,SW,4/22/2016,Economy,201,DAL-PHX
4,579,Natalie Cuevas,49,AV,5/13/2016,Economy,267,DAL-OKC
...,...,...,...,...,...,...,...,...
361,715,Justin Martin,48,SW,10/24/2017,First Class,204,DAL-ATL
362,715,Justin Martin,48,SW,11/14/2017,Economy,167,DAL-AUS
363,715,Justin Martin,48,AM,11/16/2017,First Class,81,DAL-ATL
364,715,Justin Martin,48,SW,11/24/2017,First Class,210,DAL-OKC


#### La tabla *Vuelos* tenia 400 renglones, y el join de *Vuelos* y *Pasajeros* nos dio como resultado 366 renglones. La información faltante se debe al problema que tuvimos con los Pasajeros duplicados anteriormente. 

In [28]:
vuelos_pasajeros.isnull().sum()

ID_Pasajero    0
Pasajero       0
Edad           0
Cve_LA         0
Viaje          0
Clase          0
Precio         0
Ruta           0
dtype: int64

## Exploración de Aerolíneas

In [29]:
aerolineas

,Code,Linea_Aerea
0,AA,American Airlines
1,SW,Southwest
2,AM,Aeromexico
3,AV,Avianca
4,KL,KLM


In [30]:
aerolineas= aerolineas.rename(columns = {'Code': 'Cve_LA'})
aerolineas

,Cve_LA,Linea_Aerea
0,AA,American Airlines
1,SW,Southwest
2,AM,Aeromexico
3,AV,Avianca
4,KL,KLM


#### Ahora se necesita toda la información referente a los vuelos sin importar que la información de Aerolineas quede vacía. Al ser el caso, se nos pide que se llene con 'Otra'. Es por esta razón por la que se decidió realizar un LEFT JOIN.

In [31]:
vuelos_final = pd.merge(vuelos_pasajeros,aerolineas, how="left", on= ['Cve_LA'])
vuelos_final.isnull().sum()

ID_Pasajero     0
Pasajero        0
Edad            0
Cve_LA          0
Viaje           0
Clase           0
Precio          0
Ruta            0
Linea_Aerea    15
dtype: int64

In [32]:
vuelos_final['Linea_Aerea'] = vuelos_final['Linea_Aerea'].fillna('Otra')
vuelos_final.isnull().sum()

ID_Pasajero    0
Pasajero       0
Edad           0
Cve_LA         0
Viaje          0
Clase          0
Precio         0
Ruta           0
Linea_Aerea    0
dtype: int64

vuelos_final.head()

## Análisis de Vuelos

#### Para el análisis, se solicitan únicamente las siguientes columnas: Viaje, Clase, Precio, Ruta, Edad y Linea Aerea. Se decide hacer un DataFrame extra con esos campos para no perder ningún tipo de información.

In [33]:
vuelos_analisis = vuelos_final[['Viaje','Clase','Precio','Ruta','Edad','Linea_Aerea']]
vuelos_analisis.head()

,Viaje,Clase,Precio,Ruta,Edad,Linea_Aerea
0,1/5/2016,Economy,252,DAL-SLC,60,Aeromexico
1,11/25/2016,Business,269,DAL-TUL,60,Aeromexico
2,9/6/2016,Business,186,DAL-AMA,60,American Airlines
3,4/22/2016,Economy,201,DAL-PHX,60,Southwest
4,5/13/2016,Economy,267,DAL-OKC,49,Avianca


#### Para hacer un análisis semestral de los datos, convertimos la columna "Viaje" a datetime para una mejor manipulación de ella.

In [34]:
vuelos_analisis['Viaje']= pd.to_datetime(vuelos_analisis['Viaje'], format='%m/%d/%Y')

vuelos_analisis.head()

,Viaje,Clase,Precio,Ruta,Edad,Linea_Aerea
0,2016-01-05,Economy,252,DAL-SLC,60,Aeromexico
1,2016-11-25,Business,269,DAL-TUL,60,Aeromexico
2,2016-09-06,Business,186,DAL-AMA,60,American Airlines
3,2016-04-22,Economy,201,DAL-PHX,60,Southwest
4,2016-05-13,Economy,267,DAL-OKC,49,Avianca


In [35]:
vuelos_analisis['Año'] = vuelos_analisis['Viaje'].dt.year
vuelos_analisis['Semestre'] = ['Primer' if x <= 6 else 'Segundo' for x in vuelos_analisis['Viaje'].dt.month]

### Promedio de Precio y Edad de vuelos y pasajeros agrupado por Año, Clase, Ruta, Linea Aerea y Semestre.

In [36]:
analisis= vuelos_analisis.groupby(["Año", "Clase", "Ruta", "Linea_Aerea", "Semestre"]).mean()
analisis

Precio  Edad
Año  Clase       Ruta    Linea_Aerea       Semestre              
2016 Business    DAL-AMA American Airlines Primer      58.0  64.0
                                           Segundo    186.0  60.0
                         Avianca           Primer     132.0  40.0
                         KLM               Primer     184.0  64.0
                 DAL-ATL Aeromexico        Primer     218.0  22.0
...                                                     ...   ...
2017 First Class DAL-SLC Southwest         Segundo    170.5  41.5
                 DAL-TUL Aeromexico        Primer      55.0  67.0
                                           Segundo    208.5  55.5
                         Avianca           Primer      78.0  32.0
                         KLM               Primer     242.5  50.5

[302 rows x 2 columns]

In [37]:
analisis.to_csv('AnalisisVuelos.csv')
vuelos_final.to_csv('VuelosFinal.csv')